#### Thoughts...
* Есть мультистраничные excel'и
* Одна колонка таблицы может быть объединенной
* У одной записи могут быть подзаписи с дробным номером


In [396]:
import pandas as pd
import string
import os
from glob import glob
import random
import json
import regex as re

In [17]:
path = "./soure_data/smeth_conc/smety_ishod/"
!find $path > files.txt
with open("files.txt") as f:
    files = f.readlines()
    files = list(map(lambda x: "".join(x.split('\n')), files))
allowed_ext = ('xls', 'xlsx')
excels = list(filter(lambda x: x.lower().endswith(allowed_ext), files))

In [465]:
sheet = "soure_data/smeth_conc/smety_ishod/1771608138821000208/Проектно-сметая документация (1).xlsx"
sheet = "soure_data/smeth_conc/smety_ishod/87702132064210000340000/Прил3 к ТЗ сметная докмуентация Благоустройство сквера ЦАК.xlsx"
sheet = "soure_data/smeth_conc/smety_ishod/2770182732021001448/ГБУ ТЦСО _Марьино_ филиал _Люблино_1 (1).xlsx"
#sheet = './soure_data/smeth_conc/smety_ishod/2774394681021000083/ЛС 3 (1).xls'
sheet = './weird.xlsx'
sheet = random.choice(excels)

In [466]:
sheet

'./soure_data/smeth_conc/smety_ishod/2772162044421000016/Замена резинового покрытия (2).xlsx'

In [447]:
excel_file = pd.ExcelFile(sheet)
print("Все книги:",excel_file.sheet_names)
for i in excel_file.sheet_names:
    if("смета" in i.lower()):
        choosen_name = i
        print(f"Выбрана книга: {i}")
        break
#df = pd.read_excel(sheet,header=None)
df = excel_file.parse(sheet_name=choosen_name,header=None)
df = df.dropna(how="all").dropna(axis=1,how="all")
df = df.astype(str)
df = df.reset_index(drop=True)
df.columns = range(df.columns.size)

Все книги: ['Смета СН-2012 по гл. 1-5', 'Ведомость объемов работ', 'Source', 'SourceObSm', 'SmtRes', 'EtalonRes']
Выбрана книга: Смета СН-2012 по гл. 1-5


In [448]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Smeta.RU (495) 974-1589 СН-2012 (© ОАО МЦ...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,Форма № 1а (глава 1-5),nan,nan,nan,nan,nan,nan,nan,nan,nan
2,nan,"""СОГЛАСОВАНО""",nan,nan,nan,nan,"""УТВЕРЖДАЮ""",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,nan,nan,nan,nan,nan,nan,"Директор ГБУ ""Жилищник района Покровское-Стреш...",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,nan,______________________,nan,nan,nan,nan,______________П.С.Куцев,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,nan,nan,Всего с НДС,nan,nan,nan,nan,nan,5545785.58,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
422,Составил,nan,,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
423,nan,nan,"[должность,подпись(инициалы,фамилия)]",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
424,Проверил,nan,,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [449]:
#Определение начала таблицы
def series_startswith(ser, pattern):
    index = 0
    for val in ser:
        if(val == pattern[index]):
            index+=1
        elif(val == 'nan'):
            continue
        else:
            return False
        if(index == len(pattern)):
            return True
    return False

one2elvn = [str(_) for _ in range(1,12)]
for index, row in df.iterrows():
    if(series_startswith(row, one2elvn)):
        print(index)
        table_title = index
        break

21


In [450]:
# Маппинг номеров колонок таблицы в случае объединенных excel ячеек
column_map = {}
index = 1
for ind, val in enumerate(df.iloc[table_title]):
    if(val == str(index)):
        column_map[index] = ind
        index+=1
column_map

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10}

In [451]:
# Определение разделов и подразделов
current_section = None
current_subsection = None
current_item = 0
item_indices = {0:[None,None,None,None]} # Началный индекс, Конечный индекс, раздел, подраздел
#Границы записей явлеяются отрезком [], учитывать при использовании slice

for index, row in df[table_title+1:].iterrows():
    # Определения разделов и подразделов
    if(row.str.lower().str.contains("подраздел").any()):
        if((row.str.lower().str.contains("подраздел") * row.str.lower().str.contains("итог")).any()):
            print(f"    Конец {current_subsection}")
            current_subsection = None
            item_indices[current_item][1] = index-1
        else:
            current_subsection = row[row.str.lower().str.contains("подраздел")].item()
            print(f"    Начало {current_subsection}")
    elif(row.str.lower().str.contains("раздел").any()):
        if((row.str.lower().str.contains("раздел") * row.str.lower().str.contains("итог")).any()):
            print(f"Конец {current_section}")
            current_section = None
            item_indices[current_item][1] = index-1
        else:
            current_section = row[row.str.lower().str.contains("раздел")].item()
            print(f"Начало {current_section}")
    
    # Определение номера и границ записи
    if( row[column_map[1]].isnumeric() and row[column_map[1]] == str(current_item+1)):
        if(item_indices[current_item][1] is None):
            item_indices[current_item][1] = index-1
        current_item += 1
        print(f"****Запись**** At index {index} found new item num. {current_item}")
        item_indices[current_item] = [index,None,current_section,current_subsection]

#0 запись исскуственная
del item_indices[0]

assert item_indices, "No items were found. Cannot continue"

Начало Раздел: Демонтаж покрытий
****Запись**** At index 24 found new item num. 1
****Запись**** At index 30 found new item num. 2
****Запись**** At index 39 found new item num. 3
****Запись**** At index 48 found new item num. 4
****Запись**** At index 54 found new item num. 5
****Запись**** At index 63 found new item num. 6
****Запись**** At index 72 found new item num. 7
****Запись**** At index 79 found new item num. 8
****Запись**** At index 84 found new item num. 9
****Запись**** At index 90 found new item num. 10
****Запись**** At index 94 found new item num. 11
****Запись**** At index 98 found new item num. 12
****Запись**** At index 102 found new item num. 13
Конец Раздел: Демонтаж покрытий
Начало Раздел: Демонтаж МАФ
****Запись**** At index 106 found new item num. 14
****Запись**** At index 115 found new item num. 15
****Запись**** At index 124 found new item num. 16
****Запись**** At index 133 found new item num. 17
****Запись**** At index 142 found new item num. 18
****Запись

In [452]:
item_indices

{1: [24, 29, 'Раздел: Демонтаж покрытий', None],
 2: [30, 38, 'Раздел: Демонтаж покрытий', None],
 3: [39, 47, 'Раздел: Демонтаж покрытий', None],
 4: [48, 53, 'Раздел: Демонтаж покрытий', None],
 5: [54, 62, 'Раздел: Демонтаж покрытий', None],
 6: [63, 71, 'Раздел: Демонтаж покрытий', None],
 7: [72, 78, 'Раздел: Демонтаж покрытий', None],
 8: [79, 83, 'Раздел: Демонтаж покрытий', None],
 9: [84, 89, 'Раздел: Демонтаж покрытий', None],
 10: [90, 93, 'Раздел: Демонтаж покрытий', None],
 11: [94, 97, 'Раздел: Демонтаж покрытий', None],
 12: [98, 101, 'Раздел: Демонтаж покрытий', None],
 13: [102, 103, 'Раздел: Демонтаж покрытий', None],
 14: [106, 114, 'Раздел: Демонтаж МАФ', None],
 15: [115, 123, 'Раздел: Демонтаж МАФ', None],
 16: [124, 132, 'Раздел: Демонтаж МАФ', None],
 17: [133, 141, 'Раздел: Демонтаж МАФ', None],
 18: [142, 146, 'Раздел: Демонтаж МАФ', None],
 19: [147, 152, 'Раздел: Демонтаж МАФ', None],
 20: [153, 156, 'Раздел: Демонтаж МАФ', None],
 21: [157, 160, 'Раздел: Де

In [453]:
df.iloc[item_indices[1][0]:item_indices[1][1]+1]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
24,1,2.1-3104-4-1/1,Разборка тротуаров и дорожек из плит с относко...,100 м2,0.69,nan,nan,nan,nan,nan,nan,nan,1473.36,1473.36,210.48,210.48,0.0,0.0,nan
25,nan,nan,ЗП,nan,nan,3050.44,nan,1,1,2104.8,nan,nan,nan,nan,nan,nan,nan,nan,nan
26,nan,nan,НР от ЗП,%,70,nan,nan,nan,nan,1473.36,nan,nan,nan,nan,nan,nan,nan,nan,nan
27,nan,nan,СП от ЗП,%,10,nan,nan,nan,nan,210.48,nan,nan,nan,nan,nan,nan,nan,nan,nan
28,nan,nan,ЗТР,чел-ч,18.68,nan,nan,1,nan,nan,12.889199999999999,nan,nan,nan,nan,nan,nan,nan,nan
29,nan,nan,nan,nan,nan,nan,nan,nan,3788.64,nan,5490.78,3788.64,nan,nan,nan,nan,nan,nan,nan


In [454]:
column_names = {1:"Number",
                2:"Code",
                3:"Name",
                4:"Unit",
                5:"Amount",
                6:"Price_per_unit",
                7:"Correction_coof",
                8:"Winter_coof",
                9:"Recalc_coof",
                10:"Total",
                11:"Additional"}
columns_mask = list(column_map.values())
value_mask = list(map(lambda x: x[1],filter(lambda x: x[0]>3, column_map.items())))
    
all_items = []

for item_to_parse in range(1, len(item_indices)+1):
    zap = df.iloc[item_indices[item_to_parse][0]:item_indices[item_to_parse][1]+1].reset_index(drop=True)
    #Парсинг отдельной записи
    main_work_dict = {}
    for col in range(1, 12):
        if(zap.iloc[0][column_map[col]] != "nan"):
            main_work_dict[column_names[col]] = zap.iloc[0][column_map[col]]

    main_work_dict["Sub_works"] = []
    main_work_dict["Related_works"] = []
    related_work_flag = False

    for index, row in zap[1:].iterrows():
        if(len(set(row[value_mask])) == 1 and row[value_mask[0]] == "nan"):
            continue
        if(row[column_map[3]] == 'nan'):
            continue
        if(row[column_map[1]] != 'nan' or row[column_map[2]] != 'nan'):
            if(related_work_flag):
                main_work_dict["Related_works"].append(related_work_dict)
            related_work_flag = True
            related_work_dict = {}
            related_work_dict["Sub_works"] = []
            for col in range(1, 12):
                if(row[column_map[col]] != "nan"):
                    related_work_dict[column_names[col]] = row[column_map[col]]
        elif(related_work_flag):
            sub_work_dict = {}
            for col in range(1, 12):
                if(row[column_map[col]] != "nan"):
                    sub_work_dict[column_names[col]] = row[column_map[col]]
            related_work_dict["Sub_works"].append(sub_work_dict)
        else:
            sub_work_dict = {}
            for col in range(1, 12):
                if(row[column_map[col]] != "nan"):
                    sub_work_dict[column_names[col]] = row[column_map[col]]
            main_work_dict["Sub_works"].append(sub_work_dict)
    if(related_work_flag):
        main_work_dict["Related_works"].append(related_work_dict)
        
        
    for index, row in zap[::-1].iterrows():
        first = None
        second = None
        for val in row:
            if(val != 'nan'):
                if(first):
                    second = val
                    break
                else:
                    first = val
        if(first and second and re.search('[a-zA-Zа-яА-Я]', first+second) is None):
            main_work_dict["Total"] = first
            main_work_dict["Unit_cost_with_additions"] = second
            break
            
    main_work_dict["Section"] = item_indices[item_to_parse][2]
    main_work_dict["Sub_section"] = item_indices[item_to_parse][3]
    
    all_items.append(main_work_dict)

In [455]:
with open("all_items.json", "w") as f:
    json.dump(all_items, f)
          


In [459]:
sheet


<Worksheet "Смета">

In [464]:
import pandas as pd
import numpy as np
import openpyxl
import sys,os
filename = sheet

wb = openpyxl.load_workbook(filename)
sheet = wb.active
sheet_size = 180 #sheet.max_row
c_max = 'C'+str(sheet_size)

for rowOfCellObjects in sheet['C1':c_max]:
    for cellObj in rowOfCellObjects:
        if cellObj.value != None and '-' in cellObj.value:
            #print(cellObj.coordinate, cellObj.value)
            d = str(cellObj.coordinate)[1:]
            delta = int(d)
            id = sheet['A'+d].value
            number = sheet['C'+d].value
            work_name = sheet['E'+d].value
            unit = sheet['G'+d].value
            amount = sheet['H'+d].value
            zp_ppu = sheet['J'+str(delta+1)].value                 
            em_ppu = sheet['J'+str(delta+2)].value                 
            zpm_ppu = sheet['J'+str(delta+3)].value                 
            mp_ppu = sheet['J'+str(delta+4)].value                 
            np_zp = sheet['H'+str(delta+5)].value                 
            sp_zp = sheet['H'+str(delta+6)].value                 
            np_sp_zpm = sheet['H'+str(delta+7)].value                 
            ztr_amount = sheet['H'+str(delta+9)].value                 
            total_price = sheet['O'+str(delta+11)].value                 

            print("id", id)
            print("number", number)
            print("work_name", work_name)
            print("unit", unit)
            print("amount", amount)
            print("zp_ppu", zp_ppu)
            print("em_ppu", em_ppu)
            print("zpm_ppu", zpm_ppu)
            print("mp_ppu", mp_ppu)
            print("np_zp", np_zp)
            print("sp_zp", sp_zp)
            print("np_sp_zpm", np_sp_zpm)
            print("ztr_amount", ztr_amount)
            print("total_price", total_price)
            print('==='*20)

sys.exit()

/home/chaos_ruler/.venv/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


TypeError: argument of type 'int' is not iterable